# 使用 sklearn 做单机特征工程 

转载自：http://www.cnblogs.com/jasonfreak/p/5448385.html

参考资料：

- [使用sklearn优雅地进行数据挖掘](http://www.cnblogs.com/jasonfreak/p/5448462.html)
- [特征工程](https://www.zhihu.com/question/29316149)

**特征处理**是特征工程的核心部分，sklearn 提供了较为完整的特征处理方法，包括数据预处理，特征选择，降维等。本文中使用 sklearn 中的 IRIS（鸢尾花）数据集来对特征处理功能进行说明。IRIS 数据集由 Fisher 在 1936 年整理，包含 4 个特征（Sepal.Length（花萼长度）、Sepal.Width（花萼宽度）、Petal.Length（花瓣长度）、Petal.Width（花瓣宽度）），特征值都为正浮点数，单位为厘米。目标值为鸢尾花的分类（Iris Setosa（山鸢尾）、Iris Versicolour（杂色鸢尾），Iris Virginica（维吉尼亚鸢尾））。导入IRIS数据集的代码如下：

In [5]:
from sklearn.datasets import load_iris

#导入IRIS数据集
iris = load_iris()

#特征矩阵
print(iris.data[0])

#目标向量
print(iris.target[0])

[5.1 3.5 1.4 0.2]
0


In [7]:
iris.data.shape, iris.target.shape

((150, 4), (150,))

## 数据预处理

通过特征提取，我们能得到未经处理的特征，这时的特征可能有以下问题：

- 不属于同一量纲：即特征的规格不一样，不能够放在一起比较。**无量纲化**可以解决这一问题。
- 信息冗余：对于某些定量特征，其包含的有效信息为区间划分，例如学习成绩，假若只关心“及格”或不“及格”，那么需要将定量的考分，转换成“1”和“0”表示及格和未及格。**二值化**可以解决这一问题。
- 定性特征不能直接使用：某些机器学习算法和模型只能接受定量特征的输入，那么需要将定性特征转换为定量特征。最简单的方式是为每一种定性值指定一个定量值，但是这种方式过于灵活，增加了调参的工作。通常使用哑编码（或称为 One-hot 编码）的方式将定性特征转换为定量特征：假设有 N 种定性值，则将这一个特征扩展为 N 种特征，当原始特征值为第 i 种定性值时，第 i 个扩展特征赋值为 1，其他扩展特征赋值为 0。哑编码的方式相比直接指定的方式，不用增加调参的工作，对于线性模型来说，使用哑编码后的特征可达到非线性的效果。
- 存在缺失值：缺失值需要补充。
- 信息利用率低：不同的机器学习算法和模型对数据中信息的利用是不同的，之前提到在线性模型中，使用对定性特征哑编码可以达到非线性的效果。类似地，对定量变量多项式化，或者进行其他的转换，都能达到非线性的效果。

我们使用 sklearn 中的 `preproccessing` 库来进行数据预处理，可以覆盖以上问题的解决方案。

### 无量纲化

　　无量纲化使不同规格的数据转换到同一规格。常见的无量纲化方法有标准化和区间缩放法。标准化的前提是特征值服从正态分布，标准化后，其转换成标准正态分布。区间缩放法利用了边界值信息，将特征的取值区间缩放到某个特点的范围，例如$[0, 1]$等。
  
#### 标准化

　　标准化需要计算特征的均值和标准差，公式表达为

$$
X^{'} = \frac{X - \overline{X}}{S}
$$

In [10]:
from sklearn.preprocessing import StandardScaler

#标准化，返回值为标准化后的数据
StandardScaler().fit_transform(iris.data)[0]

array([-0.90068117,  1.01900435, -1.34022653, -1.3154443 ])

#### 区间缩放法

　　区间缩放法的思路有多种，常见的一种为利用两个最值进行缩放，公式表达为：

$$
X^{'} = \frac{X - X_{min}}{X_{max} - X_{min}}
$$

In [12]:
from sklearn.preprocessing import MinMaxScaler
#区间缩放，返回值为缩放到[0, 1]区间的数据

MinMaxScaler().fit_transform(iris.data)[0]

array([0.22222222, 0.625     , 0.06779661, 0.04166667])

#### 标准化与归一化的区别

　　简单来说，标准化是依照特征矩阵的列处理数据，其通过求 z-score 的方法，将样本的特征值转换到同一量纲下。归一化是依照特征矩阵的行处理数据，其目的在于样本向量在点乘运算或其他核函数计算相似性时，拥有统一的标准，也就是说都转化为“单位向量”。

In [14]:
from sklearn.preprocessing import Normalizer
# 归一化，返回值为归一化后的数据
Normalizer().fit_transform(iris.data)[0]

array([0.80377277, 0.55160877, 0.22064351, 0.0315205 ])

### 对定量特征二值化

　　定量特征二值化的核心在于设定一个阈值，大于阈值的赋值为1，小于等于阈值的赋值为0，公式表达如下：
    
$$
X^{'} = \begin{cases}1 & X > threshold \\ 0 &X < threshold\end{cases}
$$

    使用 preproccessing 库的 `Binarizer`类对数据进行二值化的代码如下：

In [16]:
from sklearn.preprocessing import Binarizer

# 二值化，阈值设置为3，返回值为二值化后的数据
Binarizer(threshold=3).fit_transform(iris.data)[0]

array([1., 1., 0., 0.])

### 对定性特征哑编码

　　由于 IRIS 数据集的特征皆为定量特征，故使用其目标值进行哑编码（实际上是不需要的）。使用 preproccessing 库的 `OneHotEncoder` 类对数据进行哑编码的代码如下：

In [21]:
from sklearn.preprocessing import OneHotEncoder

# 哑编码，对IRIS数据集的目标值，返回值为哑编码后的数据
OneHotEncoder(categories='auto').fit_transform(iris.target.reshape((-1, 1)))

<150x3 sparse matrix of type '<class 'numpy.float64'>'
	with 150 stored elements in Compressed Sparse Row format>

### 缺失值计算

　　由于 IRIS 数据集没有缺失值，故对数据集新增一个样本，4个特征均赋值为NaN，表示数据缺失。使用preproccessing库的Imputer类对数据进行缺失值计算的代码如下：

In [25]:
from numpy import vstack, array, nan
from sklearn.impute import SimpleImputer

# 缺失值计算，返回值为计算缺失值后的数据
# 参数missing_value为缺失值的表示形式，默认为NaN
# 参数strategy为缺失值填充方式，默认为mean（均值）
SimpleImputer().fit_transform(
    vstack((array([nan, nan, nan, nan]), iris.data)))[0]

array([5.84333333, 3.05733333, 3.758     , 1.19933333])

### 数据变换

　　常见的数据变换有基于多项式的、基于指数函数的、基于对数函数的。
  
使用 preproccessing 库的 `PolynomialFeatures` 类对数据进行多项式转换的代码如下：

In [27]:
from sklearn.preprocessing import PolynomialFeatures
# 多项式转换
# 参数degree为度，默认值为2
PolynomialFeatures().fit_transform(iris.data)[0]

array([ 1.  ,  5.1 ,  3.5 ,  1.4 ,  0.2 , 26.01, 17.85,  7.14,  1.02,
       12.25,  4.9 ,  0.7 ,  1.96,  0.28,  0.04])

基于单变元函数的数据变换可以使用一个统一的方式完成，使用  preproccessing 库的 `FunctionTransformer` 对数据进行对数函数转换的代码如下：

In [28]:
from numpy import log1p
from sklearn.preprocessing import FunctionTransformer
# 自定义转换函数为对数函数的数据变换
# 第一个参数是单变元函数
FunctionTransformer(log1p).fit_transform(iris.data)[0]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


array([1.80828877, 1.5040774 , 0.87546874, 0.18232156])

## 特征选择

　当数据预处理完成后，我们需要选择有意义的特征输入机器学习的算法和模型进行训练。通常来说，从两个方面考虑来选择特征：
  
- 特征是否发散：如果一个特征不发散，例如方差接近于 0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。
- 特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择。除方差法外，本文介绍的其他方法均从相关性考虑。

　根据特征选择的形式又可以将特征选择方法分为 3 种：

- Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。
- Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。
- Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。

　我们使用 sklearn 中的 `feature_selection` 库来进行特征选择。

### Filter

#### 方差选择法

使用方差选择法，先要计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。使用 feature_selection 库的 `VarianceThreshold` 类来选择特征的代码如下：

In [31]:
from sklearn.feature_selection import VarianceThreshold
# 方差选择法，返回值为特征选择后的数据
# 参数threshold为方差的阈值
VarianceThreshold(threshold=3).fit_transform(iris.data)[0]

array([1.4])

#### 相关系数法

　　使用相关系数法，先要计算各个特征对目标值的相关系数以及相关系数的P值。用 feature_selection 库的 `SelectKBest` 类结合相关系数来选择特征的代码如下：

In [35]:
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr

# 选择K个最好的特征，返回选择特征后的数据
# 第一个参数为计算评估特征是否好的函数，
#该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
# 参数k为选择的特征个数
SelectKBest(lambda X, Y: array(list(map(lambda x: pearsonr(x, Y), X.T))).T,
            k=2).fit_transform(iris.data, iris.target)

IndexError: index 2 is out of bounds for axis 0 with size 2